In [1]:
import datashader as ds
import pandas as pd
from colorcet import fire
from datashader import transfer_functions as tf
from datashader.utils import lnglat_to_meters as webm
import pickle
import reverse_geocoder as rg
import altair as alt
import time
import numpy as np
from pathlib import Path
Path.cont = lambda x: list(os.scandir(x))
import logging
import contextlib

logging.getLogger().setLevel(logging.INFO)
@contextlib.contextmanager
def timeit(ident):
    tstart = time.time()
    yield
    elapsed = (time.time() - tstart)/60
    logging.info(f'{ident}, time: {elapsed}')

def inspect_na(df):
    print(df.isnull().sum(axis=1).value_counts(sort=False))
    df_len = len(df)
    df = df.isna().copy()
    df = df.loc[:, (df != 0).any(axis=0)]
    df = df.loc[(df!=0).any(1)]
    df_nan_dict = {c : [len(df.loc[df[c] & df[c2]]) for c2 in df.columns] for c in df.columns}
    df_nan = pd.DataFrame.from_dict(df_nan_dict)
    df_nan.set_index(df.columns, inplace=True)
    df_nan.insert(0, 'total', df_nan.max(axis=0))
    df_nan.sort_values('total', inplace=True, ascending=False)
    df_nan = pd.concat([df_nan['total'] , df_nan.iloc[:,1:][list(df_nan.index)]],axis=1)
    return (df_nan/df_len).round(2)

In [ ]:
def add_dates(df, name):
    df['date'] = name[:10]
    df['hour'] = name[11:13]
    return df

def reverse_geocode(df):
    # many thanks to https://github.com/thampiman/reverse-geocoder
    coor = tuple((lat,lon) for lat, lon in zip(df['lat'],df['lon']))
    res = rg.search(coor)
    print(res[0])
    df['country'] = [i['cc'] for i in res]
    df['provincie'] = [i['admin1'] for i in res]
    df['city'] = [i['name'] for i in res]
    return df

def process_files(path):
    dataframes = []
    for direntry in path.cont():
        if direntry.name.endswith('pickle'):
            f = open(direntry,"rb")
            df = pickle.load(f)
            df = add_dates(df, direntry.name)
            print(len(df))
            dataframes.append(df)
    return pd.concat(dataframes)

def project_gps(df):
    # Project longitude and latitude onto web mercator plane.
    df.loc[:, 'easting'], df.loc[:, 'northing'] = webm(df['lon'],df['lat'])
    return df

def categorify(df):
    categorical = ['provincie', 'country', 'hour', 'date']
    for cat in categorical: df[cat] = df[cat].astype('category')
    # categorize barometric altitude
    bins = [-1000, 2000, 10000, 100000000]
    names = ['below 2000', 'below FL100', 'above FL100']
    df['altitude'] = pd.cut(df['baroaltitude'], bins, labels=names)
    print(df['altitude'].value_counts())

    # categorize ascending/descending
    bins = [-4000000, -2, 2, 400000]
    names = ['descending', 'stable', 'ascending']
    df['mode'] = pd.cut(df['vertrate'], bins, labels=names)
    print(df['mode'].value_counts())
    return df

def save_df(df,path):
    with open(path, 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with timeit('Reading individual files files'):    
    df = process_files(Path(r'C:\Users\Jesse\Documents\GitHub\projects\flight_data_plotter\opensky_scrape'))
with timeit('Reversing geocodes'):    
    f = reverse_geocode(df) # may take a while
df = project_gps(df)
df = categorify(df)
save_df(df,Path(r'C:\Users\Jesse\Documents\GitHub\projects\flight_data_plotter\opensky_scrape\flights.db'))
print('Done, total records', len(df))

In [2]:
f = open(Path(r'C:\Users\Jesse\Documents\GitHub\projects\flight_data_plotter\opensky_scrape\flights.db'),"rb")
df = pickle.load(f)

In [3]:
def pd_remove(df, condition,description = None):
    len_before = len(df)
    df = df.loc[condition]
    len_after = len(df)
    print(f'Removing: {description}, {len_before}, {len_after}, proportion {(len_before-len_after)/len_before}') 
    return df

# drop other countries
df = pd_remove(df, df['country'] == 'NL', 'keep only NL')
df = pd_remove(df, df['baroaltitude'].notna(), 'NaN for baro altitude')
df = pd_remove(df, df['velocity'].notna(), 'NaN for velocity')
df = pd_remove(df, df['callsign'].notna(), 'NaN for callsign')
df = pd_remove(df, df['squawk'].notna(), 'NaN for squawk')

inspect_na(df)

Removing: keep only NL, 3698863, 2292411, proportion 0.38023900858182635
Removing: NaN for baro altitude, 2292411, 2277743, proportion 0.0063985035842176645
Removing: NaN for velocity, 2277743, 2242502, proportion 0.015471894766003013
Removing: NaN for callsign, 2242502, 2240289, proportion 0.0009868441588903824
Removing: NaN for squawk, 2240289, 2237332, proportion 0.001319918992594259
0    2152031
1      85301
dtype: int64


,total,geoaltitude
geoaltitude,0.04,0.04


In [4]:
empty = df[:1].reset_index()
empty[:1] = np.NaN*26

def insert_nan_rows(df, iterable):
    print(len(df))
    groups = [[groups,empty] for groups in iterable]
    print(len(groups))
    groups = [item for i in groups for item in i]
    print(len(groups))
    test = pd.concat(groups, ignore_index=True)
    print(len(test))
    return test

with timeit('sorting by icao24'):
    df2 = df.sort_values(['icao24','lastposupdate'])

groups = [split_df for split_df in np.split(df2.reset_index(drop=True), np.where(df2['lastposupdate'].diff() > 120)[0]) if len(split_df) > 20 ]
with timeit('inserting after callsign'):
    df2 = insert_nan_rows(df2, groups)

# assert len(df2.loc[df2['callsign']=='ZXP26   ']) == len(df.loc[df['callsign']=='ZXP26   '])

p = Path(r'C:\Users\Jesse\Documents\GitHub\projects\flight_data_plotter\opensky_scrape\flights_preprocessed_NL.db')
with open(p, 'wb') as handle:
    pickle.dump(df2, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('done total records', len(df2))    

INFO:root:sorting by icao24, time: 0.17105214595794677
2237332
19014
38028
INFO:root:inserting after callsign, time: 7.137673207124075
2210084
done total records 2210084


In [ ]:
p = Path(r'C:\Users\Jesse\Documents\GitHub\projects\flight_data_plotter\opensky_scrape\flights_preprocessed.db')
f = open(p),"rb")
df = pickle.load(f)

x_range = (min(df['easting']),max(df['easting']))
y_range = (min(df['northing']),max(df['northing']))

Here you can see that we have a variety of columns with data about each of the 10 million taxi trips here, such as the locations in Web Mercator coordinates, the distance, etc.  With datashader, we can choose what we want to plot on the `x` and `y` axes and see the full data immediately, with no parameter tweaking, magic numbers, subsampling, or approximation, up to the resolution of the display: